<a href="https://colab.research.google.com/github/psy794/stock_pjtt/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84%EB%A7%8C%EB%93%9C%EB%8A%94%EA%B3%BC%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [ ]:
!pip install yfinance
!pip install finance-datareader
!pip install pykrx

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import datetime
import pickle
import warnings

from pykrx import stock, bond
import yfinance as yf
import FinanceDataReader as fdr

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import matplotlib.pyplot as plt

from tqdm import tqdm

warnings.filterwarnings("ignore")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

#Data Load
#OHLCV

In [ ]:
# 데이콘에서 제공받은 데이터 로드
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [ ]:
# 각 종목별 OHLCV 데이터를 가져오기
def get_ohlcv(tickers, start_date, end_date):
    price_dataframes = []

    for ticker in tickers:
        print(f'Processing {ticker}')
        price_df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
        price_df.reset_index(inplace=True)
        price_df['종목코드'] = ticker

        price_dataframes.append(price_df)
    combined_price_df = pd.concat(price_dataframes, ignore_index=True)

    return combined_price_df

start_date = '20210601'
end_date = '20230728'   # ★★★★★★★ 여기 확인 및 수정 필요 ★★★★★★★
train_tickers = train['Ticker'].unique().tolist()  # 중복 제거하여 티커 값 리스트로 변환
train = get_ohlcv(train_tickers, start_date, end_date)

In [ ]:
import pickle
# 데이터프레임을 pickle 파일로 저장 (이유: 각 종목별 OHLCV 데이터를 가져오기가 오래걸려서 (45분정도 소요))
pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'wb'))

In [ ]:
# 저장된 pickle 파일 불러오기
loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'rb')

In [ ]:
train = loaded_train
train

#이동평균, obv, forex, 스토캐스틱, 시가총액 추가

In [ ]:
def get_mv(df):
    """
    이동평균을 계산하여 데이터프레임에 추가하는 함수

    Parameters:
        - df: 이동평균을 계산할 데이터프레임
    Returns:
        계산된 이동평균 값을 포함한 데이터프레임
    """
    grouped = df.groupby('종목코드')  # 종목별로 그룹화
    df['Close_mv5'] = grouped['종가'].rolling(5, min_periods=5).mean().reset_index(0, drop=True) # 5일 이동평균 계산
    df['Close_mv10'] = grouped['종가'].rolling(10, min_periods=10).mean().reset_index(0, drop=True)  # 10일 이동평균 계산
    df['Close_mv20'] = grouped['종가'].rolling(20, min_periods=20).mean().reset_index(0, drop=True)  # 20일 이동평균 계산

    return df

def get_obv(train):#OBV 컬럼에 추가하기
    #종목코드 2000가지를 가져오자
    stock_code = train['종목코드'].unique()

    #데이터 프레임에 틀 만들기
    train['OBV'] = 0
    train['OBV_EMA'] = 0

    for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
        df_train = train[train['종목코드'] == code]
        OBV = []
        OBV.append(0) #[-1]을 사용하기 위해 초기 값을 넣어준다.
        for i in range(1, len(df_train['종가'])): #기간 만큼 돌려준다.
            if df_train['종가'].iloc[1] > df_train['종가'].iloc[i-1]: #현재 종가가 전일 종가보다 크다면
                OBV.append(OBV[-1] + df_train['거래량'].iloc[i]) #OBV 제일 마지막 값에 현재일 거래량 값을 더한 값을 OBV에 추가.
            elif df_train['종가'].iloc[1] < df_train['종가'].iloc[i-1]: #현재 종가가 전일 종가보다 작다면
                OBV.append(OBV[-1] - df_train['거래량'].iloc[i]) #OBV 제일 마지막 값에 현재일 거래량 값을 빼준 값을 OBV에 추가.
            else: #종가가 동일하다면
                OBV.append(OBV[-1]) #OBV 제일 마지막 값을 다시 한번 OBV에 추가.

        train.loc[train['종목코드'] == code, 'OBV'] = OBV #OBV 리스트 안의 값을 코드에 넣어주자.
        train.loc[train['종목코드'] == code,'OBV_EMA'] = train[train['종목코드'] == code]['OBV'].ewm(com=20).mean() #OBV의 이동평균 값을 넣어주자.
        return train


# forex 데이터
def get_forex_index(df):
    start_date_yf = '2021-06-01'
    end_date_yf = '20230728'   # ★★★★★★★ 여기 확인 및 수정 필요 ★★★★★★★

    start_date = '20210601'
    end_date = '20230728'   # ★★★★★★★ 여기 확인 및 수정 필요 ★★★★★★★

    forex_index_data = yf.download(["USDKRW=X", "USDAUD=X", "USDJPY=X", "EURUSD=X", "CNY=X", "^GSPC", "^DJI", "^IXIC", "^STOXX50E",
                                   "^SOX",  "000001.SS", "000300.SS", "MME=F", "^TNX"],
                                  start=start_date_yf, end=end_date_yf, rounding=True)

    tmp_forex_index = forex_index_data["Close"]
    tmp_forex_index.index = pd.to_datetime(tmp_forex_index.index)
    tmp_forex_index = tmp_forex_index[(tmp_forex_index.index >= pd.to_datetime(start_date)) &
                                      (tmp_forex_index.index <= pd.to_datetime(end_date))]
    tmp_forex_index.columns = ["sse_composite_index", "csi300_index", "usdtocny", "eurtousd", "msci_emerging",
                               "usdtoaud", "usdtojpy", "usdtokrw",
                               "dow", "snp500", "nasdaq", "semicon_index", "euro50", "us10y_tsy"]

    tmp_forex_index.reset_index(drop=False, inplace=True)

    merged_df = df.merge(tmp_forex_index, left_on='날짜', right_on='Date', how='left')
    merged_df.drop(columns=['Date'], inplace=True)
    return merged_df



#스토캐스틱 fast 지표
def sto_fast_logic(df_fast):
    n = 14
    m = 5

    #n일중 최고가
    ndays_high = df_fast['고가'].rolling(window = n, min_periods = 1).max()
    #n일중 최저가
    ndays_low = df_fast['저가'].rolling(window = n, min_periods = 1).min()
    #Fast%K 계산
    fast_k = ((df_fast['종가'] - ndays_low) / (ndays_high - ndays_low)) * 100
    #Fast%D 계산
    fast_d = fast_k.ewm(span = m).mean()

    return (fast_k, fast_d)

def get_sto_fast(df):
    df['fast_k'] = 0
    df['fast_d'] = 0
    stock_code = df['종목코드'].unique()

    for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
        df_fast = df[df['종목코드'] == code]
        (fast_k, fast_d) = sto_fast_logic(df_fast)
        df.loc[df['종목코드'] == code, 'fast_k'] = fast_k
        df.loc[df['종목코드'] == code,'fast_d'] = fast_d
    return df

#스토캐스틱 slow 지표
def sto_slow_logic(df_slow):
    n = 14
    m = 5
    t = 5

    #n일중 최고가
    ndays_high = df_slow['고가'].rolling(window = n, min_periods = 1).max()
    #n일중 최저가
    ndays_low = df_slow['저가'].rolling(window = n, min_periods = 1).min()
    #Fast%K 계산
    fast_k = ((df_slow['종가'] - ndays_low) / (ndays_high - ndays_low)) * 100
    #Slow%K 계산
    slow_k = fast_k.ewm(span = m).mean()
    #Slow%D 계산
    slow_d = slow_k.ewm(span = t).mean()

    return (slow_k, slow_d)

def get_sto_slow(df):
    df['slow_k'] = 0
    df['slow_d'] = 0
    stock_code = df['종목코드'].unique()

    for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
        df_slow = df[df['종목코드'] == code]
        (slow_k, slow_d) = sto_fast_logic(df_slow)
        df.loc[df['종목코드'] == code, 'slow_k'] = slow_k
        df.loc[df['종목코드'] == code,'slow_d'] = slow_d
    return df

def add_market_cap(df):
    df['시가총액'] = 0

    stock_code = df['종목코드'].unique()

    for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
        market_cap = stock.get_market_cap('20210601', '20230728', str(code))      # ★★★★★★★ 여기 확인 및 수정 필요 ★★★★★★★
        market_cap_only = list(market_cap['시가총액'])

        df.loc[df['종목코드'] == code, '시가총액'] = market_cap_only
    return df



train = get_mv(train)
train = get_obv(train)
train = get_forex_index(train)
train = get_sto_fast(train)
train = add_market_cap(train)
train

In [ ]:
#오래걸리니까 피클로 저장
pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data_df2.pkl', 'wb'))
loaded_train_df = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data_df2.pkl', 'rb
loaded_train_df

In [ ]:
train = loaded_train_df

In [ ]:
print(train.columns)

# Index(['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '등락률', '종목코드', 'Close_mv5',
#        'Close_mv10', 'Close_mv20', 'OBV', 'OBV_EMA', 'sse_composite_index',
#        'csi300_index', 'usdtocny', 'eurtousd', 'msci_emerging', 'usdtoaud',
#        'usdtojpy', 'usdtokrw', 'dow', 'snp500', 'nasdaq', 'semicon_index',
#        'euro50', 'us10y_tsy', 'fast_k', 'fast_d', '시가총액'],
#       dtype='object')

#이후  OBV 수정하고, slow_k, slow_d 추가한 버전으로 데이터프레임 업데이트 (by재훈)

#7월20일부터 추가되는 코드 적어둘 곳 (원자재, 뉴스감성분석 등)

#업종은 마지막에 merge하면 됨 (on = '종목코드')